<a href="https://colab.research.google.com/github/itsabdullah15/Basic-ML-Projects-/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/train.csv',error_bad_lines=False, engine="python")

Skipping line 2623: unexpected end of data


In [11]:
news_dataset.shape

(2621, 5)

In [12]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id          0
title      70
author    262
text        6
label       0
dtype: int64

In [14]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [15]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [16]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
2616    Julie Hirschfeld Davis and Charlie Savage Whit...
2617    Breitbart News NYT: Dissecting Bannon’s Worldv...
2618    Scott RT’s Peter Lavelle interviews Dr. Ron Pa...
2619    Neil Irwin Yellen’s Message: My Work Here Is (...
2620    Iron Sheik TOP DEMOCRATIC DONOR: BLACKS ARE “S...
Name: content, Length: 2621, dtype: object


In [17]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [18]:
print(X)
print(Y)

        id  ...                                            content
0        0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1        1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        2  ...  Consortiumnews.com Why the Truth Might Get You...
3        3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4        4  ...  Howard Portnoy Iranian woman jailed for fictio...
...    ...  ...                                                ...
2616  2616  ...  Julie Hirschfeld Davis and Charlie Savage Whit...
2617  2617  ...  Breitbart News NYT: Dissecting Bannon’s Worldv...
2618  2618  ...  Scott RT’s Peter Lavelle interviews Dr. Ron Pa...
2619  2619  ...  Neil Irwin Yellen’s Message: My Work Here Is (...
2620  2620  ...  Iron Sheik TOP DEMOCRATIC DONOR: BLACKS ARE “S...

[2621 rows x 5 columns]
0       1
1       0
2       1
3       1
4       1
       ..
2616    0
2617    0
2618    1
2619    0
2620    1
Name: label, Length: 2621, dtype: int64


In [19]:
port_stem = PorterStemmer()

In [20]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [21]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [22]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
2616    juli hirschfeld davi charli savag white hous s...
2617    breitbart news nyt dissect bannon worldview fo...
2618    scott rt peter lavel interview dr ron paul don...
2619    neil irwin yellen messag work mostli done new ...
2620    iron sheik top democrat donor black serious f ...
Name: content, Length: 2621, dtype: object


In [23]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [24]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'scott rt peter lavel interview dr ron paul donald trump'
 'neil irwin yellen messag work mostli done new york time'
 'iron sheik top democrat donor black serious f ed head']


In [25]:
print(Y)

[1 0 1 ... 1 0 1]


In [26]:
Y.shape

(2621,)

In [27]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [28]:
print(X)

  (0, 5961)	0.2961024332067191
  (0, 5109)	0.2507784588704448
  (0, 3431)	0.344840664865904
  (0, 3328)	0.3013276950543745
  (0, 2984)	0.24575080737310007
  (0, 2721)	0.22776297620415328
  (0, 1919)	0.24419403535095047
  (0, 1484)	0.2705772945641364
  (0, 1410)	0.344840664865904
  (0, 1154)	0.2564495882931611
  (0, 969)	0.36282849603485073
  (0, 115)	0.2797888932022664
  (1, 6314)	0.29355434245101797
  (1, 2651)	0.1931141006171807
  (1, 2144)	0.7034134424434719
  (1, 1402)	0.2795255783213146
  (1, 1091)	0.20067675756431944
  (1, 877)	0.3947344010135883
  (1, 755)	0.15466707779091074
  (1, 603)	0.2891357101820222
  (2, 5941)	0.4252496208232112
  (2, 3698)	0.4955000435152344
  (2, 2330)	0.35029184869331126
  (2, 2098)	0.36884575136662984
  (2, 1216)	0.4634319287629963
  :	:
  (2618, 4302)	0.3108629498216435
  (2618, 4242)	0.2890584684992548
  (2618, 3262)	0.4398132877008483
  (2618, 2901)	0.34345879233308907
  (2618, 1675)	0.3018793712201155
  (2618, 1654)	0.2183060177819189
  (2619, 639

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [30]:
model = LogisticRegression()

In [31]:
model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [33]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9685114503816794


In [34]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [35]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9295238095238095


In [36]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [37]:
print(Y_test[3])

1
